# Ex8: 動画へのフィルタ適用

第6回，第7回に学んだ画像を変換する処理を動画に適用してみましょう．

- 変換したい動画を用意してください．
- 動画を変換するfilter_funcを実装してください．

## 必要なライブラリのインストール

In [22]:
# プログレスバー用ライブラリのインストール
from fastprogress import progress_bar

# 動画入出力に使うライブラリ
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Colab上での出力用ライブラリ
from IPython.display import HTML
import base64
import io

## 表示・入出力用の補助関数

In [23]:
def display_video(video_file):
    cap = cv2.VideoCapture(video_file)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    video = io.open(video_file, 'r+b').read()
    encoded = base64.b64encode(video)
    html_code = '''<video width="70%" height="70%" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''

    return(HTML(data=html_code.format(encoded.decode('ascii'))))


def read_scale(cap, w, h):
    ret, I = cap.read()
    if ret == False:
        return False, None

    I = cv2.resize(I, dsize=(w, h))
    return ret, I


## 動画ファイルの読み込み

- フィルタ処理をかけたい動画をin_fileに指定してください．
- 下に動画の幅，高さ，fps，フレーム数などの情報が表示されます．

In [24]:
# @markdown Work: 自分のファイル名に置き換えてください．
in_file = "xylophone.mp4"  # @param {type:"string"}

codec = "vp80"

# 動画の読み込み，属性情報の取得
cap = cv2.VideoCapture(in_file)

num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

print("Video Info: {0}".format(in_file))
print("- (w, h): ({0}, {1})".format(w, h))
print("- fps: {0}".format(fps))
print("- num_frames: {0}".format(num_frames))


Video Info: xylophone.mp4
- (w, h): (320, 240)
- fps: 30.0
- num_frames: 9


## Work: フィルタ処理の実装

以下の動画出力ループ内で動画にフィルタ処理をかけてください．


```
# 動画出力ループ
for i in progress_bar(range(num_frames)):
  # 各フレームの読み取り処理
  ret, I = read_scale(cap, w_out, h_out)
  ...

  # Work: フレーム毎のフィルタ処理の実装
  F = filter_func(I)

  # 変換画像の書き込み
  writer.write(F)

...
```

- filter_funcは画像IをFに変換する処理であれば自由に設定してよいです．
- 第6回，第7回の授業を参考にしつつ，自分でフィルタ処理をデザインしてください．


In [93]:
def lambda1(x):
    return x * 0.5

def filter_func1(I):
    F = np.array(I)
    F[:, :,:1] = 0.0
    return F

def filter_func2(I):
    F = np.array(I)
    print(len(F[0]),len(F),len(F[0,0]))
    i = len(F)//2
    j = len(F[0])//2
    k = len(F[0,0])//2
    F[:i,:j,:] = 0.0
    return F
#F = np.array(I)
#    blur = np.vectorize(lambda x:x * 0.3)
#    F = blur(F)

## 出力動画の確認

filter_func1, fiter_func2が実装できていれば，フィルタで変換した動画が出力されます．

In [94]:
#@title 入力ファイルの設定 

#@markdown 以下のフレーム数の処理に限定（動画の一部分だけ処理）
target_frames = 300 #@param {type:"slider", min:24, max:600, step:1}
num_frames = min(num_frames, target_frames)

#@markdown 大きいと重いので横幅をw_targetに縮小．
w_target = 512 #@param {type:"slider", min:128, max:800, step:1}

#@markdown filter_func1とfilter_func2のどちらで処理するか．
func_choice = "filter_func1" #@param ["filter_func1", "filter_func2"]
out_file = "{0}.webm".format(func_choice)

filter_func = filter_func1
if func_choice == "filter_func2":
  filter_func = filter_func2

cap = cv2.VideoCapture(in_file)

num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

w_out = w
h_out = h

scale = w_target / w
if scale < 1.0:
  w_out = int(w*scale)
  h_out = int(h*scale)

fourcc = cv2.VideoWriter_fourcc(*codec)
writer = cv2.VideoWriter(out_file, fourcc, fps, (w_out, h_out))

# 動画出力ループ
for i in progress_bar(range(num_frames)):
  # 各フレームの読み取り処理
  ret, I = read_scale(cap, w_out, h_out)
  
  if ret==False:
    break

  # 実装したフィルタ関数の適用．
  F = filter_func(I)

  # 変換画像の書き込み
  writer.write(F)

# 動画オブジェクトの解放
cap.release()
writer.release()

# 出力動画の表示  
display_video(out_file)

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'
